# Contents

1. EDA
  1. 데이터 확인
  1. Correlation 분석
      1. heatmap
      1. pairplot
      1. VIF
1. rough modeling
  - 결과 진단
      1. 잔차 정규성 검정
          - plot (residual - feature)
      1. 이분산 검정
          - scatter plot (residual - feature)
      1. 자기상관 검정
          - PCA 결정
      1. target-feature check
          1. partial regression plot
          1. ccpr plot
1. 컬럼별 최적 정규화
1. modeling
  1. model with outlier
  1. model w/o outlier
  1. 결과 진단
      1. 잔차 정규성 검정
      1. 이분산 검정
      1. 자기상관 검정
      1. target-feature check
1. Cross validation 결과
1. Kaggle 결과
1. 결론

---

In [1]:
%matplotlib inline
from taxi_pakage import *

# EDA

### Data check

In [3]:
train = pd.read_csv('train.csv')

### Correalation

#### heatmap

#### VIF

#### pairplot
- 관계 높은 애들만

# Rough Modeling

# Optimal regulize

# Modeling

# Cross validation

# Kaggle result

# Conclusion